#Permite visualizar los box de un XML asociado a una imagen para probar que sean correctos:


1) Cargar librerías:

In [ ]:
from IPython.display import Image, display
from PIL import Image as ImPIL
import numpy as np
import os
import cv2
import os.path

import tensorflow as tf
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import xml.etree.cElementTree as ET

print ("Librerías cargadas.")

2) Montar el Drive:

In [ ]:
# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

3) Llevar a cabo la prueba:

In [ ]:
#@title define parámetros a utilizar { run: "auto" }

mostrarImagenOriginal = False #@param {type:"boolean"}

extraerObjetos = False #@param {type:"boolean"}

mostrarImagenRecuadros = True #@param {type:"boolean"}

In [ ]:
# define la carpeta donde están las imágenes para procesar
drive_path = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas'
data_dir_path = drive_path + '/Cursogramas/Generados'

# levanta los XML  para procesar
all_xml_array = [ fn for fn in os.listdir( data_dir_path ) if fn.endswith('.xml')  ]

print("> XML a probar: ", len(all_xml_array))

In [ ]:
# desordena y toma algunos al azar
np.random.shuffle(all_xml_array)
all_xml_array, _ = np.split(np.array(all_xml_array), [10])

print("> XML a probar: ", len(all_xml_array))

In [ ]:
# arma lista de colores para los recuados
colors = list(ImageColor.colormap.values())
colors = sorted(colors, key=lambda _: _[1])

for xml_file in all_xml_array:

    print("\n\n> ", xml_file)

    # carga la info del XML original
    et = ET.parse(data_dir_path + '/' + xml_file)
    element = et.getroot()
    element_objs = element.findall('object') 
    element_filename = element.find('filename').text
    ##if not('.png' in element_filename):
    ##    element_filename  = element_filename + '.png'
    img_filename = os.path.join(data_dir_path, element_filename)
    
    # carga la imagen
    imageCargada = ImPIL.open(img_filename)
    if mostrarImagenOriginal: 
        display( imageCargada )

    # imagen auxiliar para mostrar recuadros
    img = tf.io.read_file(img_filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = img.numpy()
    image_pil = Image.fromarray(np.uint8(img)).convert("RGB")
    draw = ImageDraw.Draw(image_pil)
    im_width, im_height = image_pil.size

    # procesa los elementos en el archivo XML original para generar el nuevo
    for element_obj in element_objs:

        # obtiene la información actual de la imagen
        class_name = element_obj.find('name').text 

        # obtiene info del box actual
        obj_bbox = element_obj.find('bndbox')
        x1 = int(round(float(obj_bbox.find('xmin').text)))
        y1 = int(round(float(obj_bbox.find('ymin').text)))
        x2 = int(round(float(obj_bbox.find('xmax').text)))
        y2 = int(round(float(obj_bbox.find('ymax').text)))

        if extraerObjetos:
            # extrae los objetos de la imagen
            print("  --- ", class_name)  
            imBox = imageCargada.crop((x1,y1,x2,y2))
            display( imBox )

        if mostrarImagenRecuadros:
            # genera los recuadros correspondientes
            color = colors[(hash(class_name)) % len(colors)]
            if color == "#ffffff":
              color = "#000000"
            draw.line([(x1, y1), (x1, y2), (x2, y2), (x2, y1), (x1, y1)], width=4, fill=color)
             

    if mostrarImagenRecuadros:
       # muestra la imagen con los recuadros correspondientes
       ##np.copyto(img, np.array(image_pil))
       imMostrar = Image.fromarray(np.array(image_pil), 'RGB')
       display( imMostrar )
